<div class="alert alert-block alert-warning">
### Table of Content 
 
This jupyter notebook provides the python code for training the predictive model described in the paper "Forecasting Airport Transfer Passenger Flow Using Real-Time Data and Machine Learning".
<br>
<br>
[Functions required in model fitting](#Functions-required-in-model-fitting)<br>
[Cross validation to find optimal tuning parameters](#Cross-validation-to-find-optimal-tuning-parameters)<br>
[Fit the model to the entire training set](#Fit-the-model-to-the-entire-training-set)<br>
[Visualize the $stable$ tree](#Visualize-the-$stable$-tree)<br>
[Find the segment that each passenger in the test set belongs to](#Find-the-segment-that-each-passenger-in-the-test-set-belongs-to)

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from scipy.stats import gamma
from io import StringIO
import pydotplus
import matplotlib.pyplot as plt
from IPython.display import Image

## Functions required in model fitting

Functions for fitting Gamma distributions to the leaves (gammaDistribution), predicting quantiles from the regression tree (predQuantile), and calculating the pinball score (measurePinball).

In [ ]:
def gammaDistribution(model,DummDf):
    indexes = model.apply(DummDf.drop('Delta', axis=1))
    DeltasDf = DummDf[['Delta']]
    DeltasDf['leafs']=model.apply(DummDf.drop('Delta', axis=1))
    params = {'leaf':[], 'a':[], 'scale':[]}
    for i, ind in enumerate(set(indexes)):
        temp = DeltasDf[DeltasDf.leafs==ind]    
        coefs = gamma.fit(temp.Delta, floc=0)
        params['leaf'].append(ind)
        params['a'].append(coefs[0])
        params['scale'].append(coefs[2])
    paramDf=pd.DataFrame(params)   
    return paramDf

def predQuantile(model,para,quantile,DummPred):
    leaf = model.apply(DummPred)
    leaf = pd.DataFrame({'leaf':leaf})
    para = para.set_index('leaf')
    paxPara = pd.merge(leaf, para, left_on='leaf', right_index=True, how='left', sort=False)
    n = paxPara.shape[0]-1
    prediction = np.zeros((1,n+1))
    a = paxPara['a']
    scale = paxPara['scale']
    prediction[0] = gamma.ppf(quantile,a,loc=0,scale=scale)
    return(prediction)

def measurePinball(actual,quant,per):
    n = quant.shape[1] - 1
    pinball = np.zeros((1,n+1))
    I = (actual > quant[0])*1
    pinball[0] = (actual-quant[0])*per*I + (quant[0] - actual)*(1-per)*(1-I)
    return pinball.sum()/(n+1)

## Cross validation to find optimal tuning parameters

Here we only provide code for running one round of cross-validation. The tree is fit — for a range of values of the two parameters — to 80% of the data in the small training set, and the pinball loss of the 0.05, 0.25, 0.50, 0.75 and 0.95 quantiles are computed and then averaged in the remaining 20%. 

This process should be repeated $n$ times for $n$-fold cross-validation, using different small training and validation sets. The $n$ average pinball scores are then averaged to find the optimal tuning parameters. 

In [ ]:
# load in the data
df = pd.read_csv('trainingSet_f1.csv')
df_val = pd.read_csv('validationSet_f1.csv')

In [ ]:
# In the training set, remove variables that are not useful
df = df.drop('ib_aircraft_type',1)
df = df.drop('ob_aircraft_type',1)
df = df.drop('ob_aircraft_class',1)
df = df.drop('ib_aircraft_class',1)

# produce dummy variables for categorical variables
DummDf = pd.get_dummies(df,sparse=False)
DummDf = DummDf.drop("ib_terminal_I",1)
DummDf = DummDf.drop("passenger_travel_class_EC",1)
DummDf = DummDf.drop("ib_aircraft_body_N",1)
DummDf = DummDf.drop("ib_stand_type_P",1)
DummDf = DummDf.drop("ob_aircraft_body_N",1)
DummDf = DummDf.drop("ob_stand_type_P",1)

In [ ]:
# In the validation set, remove variables that are not useful
df_val = df_val.drop('ib_aircraft_type',1)
df_val = df_val.drop('ob_aircraft_type',1)
df_val = df_val.drop('ob_int_dom',1)
df_val = df_val.drop('ob_aircraft_class',1)
df_val = df_val.drop('ib_aircraft_class',1)

# produce dummy variables for categorical variables
DummValDf = pd.get_dummies(df_val,sparse=False)
actual = DummValDf['Delta']
DummValDf = DummValDf.drop("Delta",1)
DummValDf = DummValDf.drop("ib_terminal_I",1)
DummValDf = DummValDf.drop("passenger_travel_class_EC",1)
DummValDf = DummValDf.drop("ib_aircraft_body_N",1)
DummValDf = DummValDf.drop("ib_stand_type_P",1)
DummValDf = DummValDf.drop("ob_aircraft_body_N",1)
DummValDf = DummValDf.drop("ob_stand_type_P",1)

In [ ]:
# delete df and df_val to save some space
del df
del df_val

In [ ]:
# Set up the grids
maxDepth = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
minNodeSize = [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500]

# Set up matrices to store the pinball losses
pinball_05 = np.zeros((16,15))
pinball_95 = np.zeros((16,15))
pinball_25 = np.zeros((16,15))
pinball_75 = np.zeros((16,15))
pinball_50 = np.zeros((16,15))

In [ ]:
# Run loops to calcuate pinball losses for different settings of the tuning parameters
for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    for j in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
        tree = DecisionTreeRegressor(max_depth = maxDepth[i], min_samples_leaf = minNodeSize[j], random_state = 687)
        model = tree.fit(DummDf.drop('Delta', axis=1), DummDf.Delta)
        pred =  tree.predict(DummValDf)
        para = gammaDistribution(model,DummDf)
        pred_05 = predQuantile(model,para,0.05,DummValDf)
        pred_25 = predQuantile(model,para,0.25,DummValDf)
        pred_50 = predQuantile(model,para,0.50,DummValDf)
        pred_75 = predQuantile(model,para,0.75,DummValDf)
        pred_95 = predQuantile(model,para,0.95,DummValDf) 
        pinball_05[i][j] = measurePinball(actual,pred_05,0.05)
        pinball_25[i][j] = measurePinball(actual,pred_25,0.25)
        pinball_75[i][j] = measurePinball(actual,pred_75,0.75)
        pinball_95[i][j] = measurePinball(actual,pred_95,0.95)
        pinball_50[i][j] = measurePinball(actual,pred_50,0.50)
        del model; del pred; del para; del pred_25; del pred_75; del pred_95; del pred_05 

In [ ]:
# Calculate the average pinball loss
pinball_average <- (pd.DataFrame(pinball05)
                    +pd.DataFrame(pinball25)
                    +pd.DataFrame(pinball50)
                    +pd.DataFrame(pinball75)
                    +pd.DataFrame(pinball95))/5

In our study, we repeat the above process for 5 times, and compute the average values of "pinball_average". We find that setting max_depth and min_samples to 15 and 700 gives us the lowset average pinball loss.

## Fit the model to the entire training set

Retrain the model to the entire training data. Set the tuning parameters, the maxmum depth of the tree and the minimum node size, to the optimal values obtained from cross validation (15 and 700 respectively). 

In [ ]:
df = pd.read_csv('trainingSet.csv')

In [ ]:
# In the training set, remove variables that are not useful
df = df.drop('ib_aircraft_type',1)
df = df.drop('ob_aircraft_type',1)
df = df.drop('ob_aircraft_class',1)
df = df.drop('ib_aircraft_class',1)

# produce dummy variables for categorical variables
DummDf = pd.get_dummies(df,sparse=False)
DummDf = DummDf.drop("ib_terminal_I",1)
DummDf = DummDf.drop("passenger_travel_class_EC",1)
DummDf = DummDf.drop("ib_aircraft_body_N",1)
DummDf = DummDf.drop("ib_stand_type_P",1)
DummDf = DummDf.drop("ob_aircraft_body_N",1)
DummDf = DummDf.drop("ob_stand_type_P",1)

In [ ]:
# fit the tree using the optimal tuning parameters
tree = DecisionTreeRegressor(max_depth = 15, min_samples_leaf = 700, random_state = 687)
model = tree.fit(DummDf.drop('Delta', axis=1), DummDf.Delta)

# get the gamma distribution for each leaf by calling the gammaDistribution function.
para = gammaDistribution(model,DummDf)

In [ ]:
# save the tree model to a pickle file for later use
with open('treeModel.pickle', 'wb') as f:
    pickle.dump(model, f, 4)
# save the parameters of the gamma distribution for later use
para.to_csv('coef.csv',index=False)

In [ ]:
# Calculate feature importance values
features=[pair for pair in zip(DummDf.drop(['Delta'],axis=1).columns,model.feature_importances_) if pair[1]>=0]
pd.DataFrame(features,columns=['feature', 'importance']).sort_values(by='importance',ascending=False)

## Visualize the $stable$ tree

Next, we visualize the $stable$ tree (i.e. the first four levels of the tree) defined in the paper. 

In [ ]:
# Create a string buffer dot_data 
dot_data = StringIO()
export_graphviz(model, out_file = dot_data, feature_names = DummDf.drop('Delta', axis=1).columns,rounded = True,  
                      proportion = True, rotate = 0, filled = True, node_ids=True, max_depth = 3)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 

# visualize the tree
Image(graph.create_png())

## Find the segment that each passenger in the test set belongs to 

In [ ]:
# load in the test set
df = pd.read_csv('testingSet.csv')

In [ ]:
# in the training set, remove variables that are not useful
df = df.drop('ib_aircraft_type',1)
df = df.drop('ob_aircraft_type',1)
df = df.drop('ob_aircraft_class',1)
df = df.drop('ib_aircraft_class',1)

# produce dummy variables for categorical variables
DummDf = pd.get_dummies(df,sparse=False)
DummDf = DummDf.drop("ib_terminal_I",1)
DummDf = DummDf.drop("passenger_travel_class_EC",1)
DummDf = DummDf.drop("ib_aircraft_body_N",1)
DummDf = DummDf.drop("ib_stand_type_P",1)
DummDf = DummDf.drop("ob_aircraft_body_N",1)
DummDf = DummDf.drop("ob_stand_type_P",1)

In [ ]:
# record the leaf number for each passenger in the test set
passengers['leaf'] = model.apply(DummDf)

# save the leaf numbers in a CSV file for later use
passengers['leaf'].to_csv('leaf-testing.csv')